In [1]:
import cv2
import requests
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub

In [2]:
# Pre-trained object detection model
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")

In [3]:
class_to_ingredient = {
    1: "tomato",
    2: "onion",
    3: "potato",
    4: "garlic",
    5: "ginger",
    6: "green chili",
    7: "coriander leaves (cilantro)",
    8: "mint leaves",
    9: "curry leaves",
    10: "spinach (palak)",
    11: "paneer",
    12: "cumin seeds",
    13: "mustard seeds",
    14: "turmeric powder",
    15: "red chili powder",
    16: "garam masala",
    17: "coriander powder",
    18: "fennel seeds",
    19: "fenugreek seeds",
    20: "cardamom",
    21: "cloves",
    22: "cinnamon",
    23: "bay leaves",
    24: "asafoetida (hing)",
    25: "ghee",
    26: "rice",
    27: "wheat flour",
    28: "biryani masala",
    29: "chicken",
    30: "mutton",
    31: "fish",
    32: "eggs",
    33: "chana dal",
    34: "toor dal",
    35: "moong dal",
    36: "masoor dal",
    37: "urad dal",
    38: "coconut",
    39: "coconut milk",
    40: "jaggery",
    41: "tamarind",
    42: "lemon",
    43: "yogurt (curd)",
    44: "buttermilk",
    45: "milk",
    46: "cream",
    47: "cashews",
    48: "almonds",
    49: "raisins",
    50: "bitter gourd (karela)",
    51: "brinjal (eggplant)",
    52: "cauliflower",
    53: "capsicum (bell pepper)",
    54: "ladyfinger (okra)",
    55: "cabbage",
    56: "carrot",
    57: "green peas",
    58: "methi leaves (fenugreek)",
    59: "mooli (radish)",
    60: "gourd (lauki)",
    61: "pumpkin",
    62: "kala chana (black chickpeas)",
    63: "rajma (kidney beans)",
    64: "black gram (urad dal whole)",
    65: "poha (flattened rice)",
    66: "sooji (semolina)",
    67: "besan (gram flour)",
    68: "sabudana (tapioca pearls)",
    69: "mustard oil",
    70: "vegetable oil",
    71: "saffron",
    72: "poppy seeds",
    73: "whole wheat",
    74: "puffed rice",
    75: "pickle",
    76: "papad",
    77: "jowar",
    78: "bajra",
    79: "amchur (dried mango powder)",
    80: "khoya",
    81: "makhana (fox nuts)",
    82: "tandoori masala",
    83: "chaat masala",
    84: "pav bhaji masala",
    85: "sambar powder",
    86: "rasam powder",
    87: "idli rice",
    88: "black salt",
    89: "kasuri methi (dried fenugreek leaves)",
    90: "imli (tamarind)",
    91: "soya chunks",
    92: "gond (edible gum)",
    93: "badam (almond)",
    94: "kaju (cashew)",
    95: "dahi (curd)"
}


In [4]:
def capture_image():
    # Access webcam
    cap = cv2.VideoCapture(0)  # 0 is usually the primary webcam

    while True:
        ret, frame = cap.read()
        if ret:
            # Show the frame
            cv2.imshow('Capture Image', frame)
            
            # Press 's' to save the captured image
            if cv2.waitKey(1) & 0xFF == ord('s'):
                img_name = "captured_image.jpg"
                cv2.imwrite(img_name, frame)
                print(f"Image saved as {img_name}")
                break

    cap.release()
    cv2.destroyAllWindows()

    return img_name

In [5]:
def detect_ingredients(image_path):
    # Load and preprocess image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = tf.image.resize(image_rgb, (512, 512))
    image_resized = np.expand_dims(image_resized, axis=0)
    
    # Detect objects
    detections = detector(image_resized)
    
    # Extract detection classes and confidence scores
    class_ids = detections['detection_classes'][0].numpy().astype(int)
    scores = detections['detection_scores'][0].numpy()
    # Filter detected ingredients by a confidence threshold
    threshold = 0.5
    detected_ingredients = [class_to_ingredient[c] for c, s in zip(class_ids, scores) if s > threshold]
    
    return detected_ingredients

In [6]:
def get_recipes(ingredients):
    api_key = '7e5a62c0114144bebabeefcdaacc68ac'  
    url = f"https://api.spoonacular.com/recipes/findByIngredients?ingredients={','.join(ingredients)}&number=5&apiKey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        recipes = [recipe['title'] for recipe in data]
        return recipes
    else:
        print("Error fetching recipes from API:", response.status_code)
        return []

In [ ]:
# Capture image from webcam
image_path = capture_image()

# Detect ingredients from the captured image
detected_ingredients = detect_ingredients(image_path)

if detected_ingredients:
    print("Detected Ingredients:", detected_ingredients)
    
    # Fetch recipes based on detected ingredients
    recipes = get_recipes(detected_ingredients)
    
    if recipes:
        print("\nHere are some recipe suggestions based on your ingredients:")
        for i, recipe in enumerate(recipes, 1):
            print(f"{i}. {recipe}")
    else:
        print("No recipes found.")
else:
    print("No ingredients detected in the image.")

In [ ]:
# Assume 'image' is captured from the camera and preprocessed
prediction = model.predict(image)
print("Predicted Ingredient:", class_to_ingredient[np.argmax(prediction)])